<a href="https://colab.research.google.com/github/fulltrick/-/blob/main/waifast7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================================
#  Stable Diffusion WebUI (A1111) を Python 3.10 で起動する版 - waifast7
#  - すべての処理を1セルで完結
#  - Share URLを自動表示 + 音声通知
#  - Colab カーネルは 3.11/3.12 でもOK（実行系だけ 3.10）
#  - xFormers 未使用（--opt-sdp-attention でSDPA）
#  - ControlNet (SDXL) / Civitai 取得ロジックを踏襲
# ==========================================================

print("🧪 Notebook kernel Python (参考):")
!python -V

import os, sys, json, shutil, subprocess, time, re
import numpy as np
from google.colab import drive, userdata
from IPython.display import Audio, display, HTML

# ----------------------------
# 0) Micromamba で Python 3.10 環境を用意
# ----------------------------
print("\n📦 micromamba をセットアップして py310 環境を作成...")
MICROMAMBA = "/usr/local/bin/micromamba"
if not os.path.exists(MICROMAMBA):
    !wget -qO /tmp/micromamba.tar.bz2 https://micromamba.snakepit.net/api/micromamba/linux-64/latest
    !tar -xjf /tmp/micromamba.tar.bz2 -C /usr/local/bin bin/micromamba --strip-components=1

!micromamba create -y -n py310 -c conda-forge python=3.10.13 pip git aria2 curl

print("\n🧪 Runtime (py310) バージョン確認：")
!micromamba run -n py310 python -V

# ----------------------------
# 1) システム側の最低限ツール & Drive
# ----------------------------
print("\n🔧 apt / Google Drive 準備...")
!apt-get update -qq
!apt-get install -y -qq aria2 wget curl

try:
    drive.mount('/content/drive')
    print("✅ Google Drive をマウントしました")
except Exception as e:
    print(f"⚠️ Drive マウント失敗または既にマウント済み: {e}")

drive_base_data_path = '/content/drive/MyDrive/sd_colab_data'
drive_models_path     = os.path.join(drive_base_data_path, 'models/Stable-diffusion')
drive_controlnet_path = os.path.join(drive_base_data_path, 'models/ControlNet')
drive_vae_path        = os.path.join(drive_base_data_path, 'models/VAE-approx')
os.makedirs(drive_models_path, exist_ok=True)
os.makedirs(drive_controlnet_path, exist_ok=True)
os.makedirs(drive_vae_path, exist_ok=True)

# ----------------------------
# 2) 既存WebUI削除 → クローン
# ----------------------------
webui_path = '/content/stable-diffusion-webui'
if os.path.exists(webui_path):
    shutil.rmtree(webui_path)
    print(f"🧹 既存のWebUIを削除: {webui_path}")

print("\n⬇️ Stable Diffusion WebUI をクローン...")
%cd /content
!git clone -q https://github.com/AUTOMATIC1111/stable-diffusion-webui.git

# ----------------------------
# 3) PyTorch (cu121) を「py310環境」にインストール
# ----------------------------
print("\n🔥 PyTorch (cu121) を py310 環境へインストール...")
!micromamba run -n py310 python -m pip -q install --upgrade pip
!micromamba run -n py310 pip install -q --index-url https://download.pytorch.org/whl/cu121 torch torchvision torchaudio

print("\n📚 追加ライブラリ（py310 環境）をインストール...")
!micromamba run -n py310 pip install -q open_clip_torch transformers timm requests

# ----------------------------
# 4) 拡張（ControlNet / Civitai Helper）
# ----------------------------
print("\n🔌 拡張機能インストール...")
%cd /content/stable-diffusion-webui/extensions
!git clone -q https://github.com/Mikubill/sd-webui-controlnet.git
!git clone -q https://github.com/butaixianran/Stable-Diffusion-Webui-Civitai-Helper.git
print("✅ 拡張インストール完了")

# ----------------------------
# 5) Civitai API トークン & モデル取得
# ----------------------------
print("\n🔑 Civitai API トークンを取得（Colab 左の🔑から事前登録）...")
try:
    CIVI = userdata.get('CIVITAI_API_TOKEN')
    if not CIVI or not CIVI.strip():
        raise ValueError("CIVITAI_API_TOKEN が未設定")
    os.environ['CIVITAI_API_TOKEN'] = CIVI.strip()
    print("✅ CIVITAI_API_TOKEN を検出")
except Exception as e:
    print("❌ Civitai APIトークンが見つかりません")
    print("   → サイドバーの 🔑 Secrets で `CIVITAI_API_TOKEN` を設定して再実行してください。")
    raise

def get_latest_model_version_id(token: str):
    print("🔍 WAI-NSFW-illustrious-SDXL の最新バージョンIDを取得中...")
    model_id = "827184"
    api_url  = f"https://civitai.com/api/v1/models/{model_id}?token={token}"
    r = subprocess.run(['curl','-s',api_url], capture_output=True, text=True)
    if r.returncode != 0 or not r.stdout:
        print("⚠️ API呼び出しエラー、既知IDへフォールバックします")
        return None
    try:
        data = json.loads(r.stdout)
        mv = data.get('modelVersions') or []
        if mv:
            vid = str(mv[0]['id'])
            name = mv[0].get('name','Unknown')
            print(f"✅ 最新バージョン: {name} (ID: {vid})")
            return vid
    except Exception:
        pass
    print("⚠️ モデルバージョン情報が取得できずフォールバックします")
    return None

def download_target_model(token: str, version_id: str):
    print(f"🔄 モデルをダウンロード/配置: version_id={version_id}")
    model_dir_local = '/content/stable-diffusion-webui/models/Stable-diffusion'
    os.makedirs(model_dir_local, exist_ok=True)
    output_filename   = "waiNSFWIllustrious_latest.safetensors"
    output_path_drive = f"{drive_models_path}/{output_filename}"
    output_path_local = f"{model_dir_local}/{output_filename}"

    if os.path.exists(output_path_drive) and os.path.getsize(output_path_drive) > 1_000_000:
        print(f"✅ Driveに既存 → ローカルへコピー")
        shutil.copy(output_path_drive, output_path_local)
        return

    print("⬇️ Civitai から取得中...")
    download_url = f"https://civitai.com/api/download/models/{version_id}?token={token}"
    r = subprocess.run([
        'aria2c','-c','-x16','-s16','--check-certificate=false',
        '--summary-interval=10','--content-disposition-default-utf8=true',
        download_url, '-d', drive_models_path, '-o', output_filename
    ], capture_output=True, text=True)
    if r.returncode != 0:
        print("❌ ダウンロード失敗:\n", r.stderr or r.stdout)
        raise SystemExit(1)

    if os.path.exists(output_path_drive) and os.path.getsize(output_path_drive) > 1_000_000:
        shutil.copy(output_path_drive, output_path_local)
        print("✅ モデル配置完了:", output_path_local)
    else:
        print("❌ Drive 上のファイルが見当たらない/不完全")
        raise SystemExit(1)

latest_vid = get_latest_model_version_id(os.environ['CIVITAI_API_TOKEN'])
if latest_vid is None:
    print("💡 既知のバージョンIDでフォールバック試行")
    for vid in ["1410435","1396035","1378467","1360950"]:
        try:
            download_target_model(os.environ['CIVITAI_API_TOKEN'], vid)
            break
        except SystemExit:
            continue
else:
    download_target_model(os.environ['CIVITAI_API_TOKEN'], latest_vid)

# ----------------------------
# 6) ControlNet (SDXL) モデル
# ----------------------------
print("\n🔄 ControlNet (SDXL) モデルダウンロード...")
controlnet_dir_local = '/content/stable-diffusion-webui/extensions/sd-webui-controlnet/models'
os.makedirs(controlnet_dir_local, exist_ok=True)

controlnet_models = [
    {
        "name": "OpenPoseXL2.safetensors",
        "url":  "https://huggingface.co/thibaud/controlnet-openpose-sdxl-1.0/resolve/main/OpenPoseXL2.safetensors"
    },
    {
        "name": "diffusers_xl_canny_full.safetensors",
        "url":  "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_full.safetensors"
    }
]

for m in controlnet_models:
    name, url = m['name'], m['url']
    path_drive = f"{drive_controlnet_path}/{name}"
    path_local = f"{controlnet_dir_local}/{name}"
    if os.path.exists(path_drive) and os.path.getsize(path_drive) > 1_000_000:
        print(f"✅ Drive 既存 {name} → ローカルへコピー")
        shutil.copy(path_drive, path_local)
    else:
        print(f"⬇️ {name} を取得中（Hugging Face）...")
        r = subprocess.run(['aria2c','-c','-x8','-s8', url, '-d', drive_controlnet_path, '-o', name],
                           capture_output=True, text=True)
        if r.returncode == 0 and os.path.exists(path_drive) and os.path.getsize(path_drive) > 1_000_000:
            shutil.copy(path_drive, path_local)
            print(f"✅ {name} 配置完了")
        else:
            print(f"⚠️ {name} の取得に失敗（続行）")

print("✅ ControlNet モデルの準備完了")

# ----------------------------
# 7) WebUI 設定ファイル
# ----------------------------
print("\n⚙️ WebUI 設定ファイルを作成...")
config_content = """{
  "samples_save": true,
  "samples_format": "png",
  "grid_save": true,
  "grid_format": "png",
  "enable_pnginfo": true,
  "save_selected_only": true,
  "jpeg_quality": 80,
  "export_for_4chan": true,
  "img_downscale_threshold": 4.0,
  "target_side_length": 4000,
  "img_max_size_mp": 200,
  "clean_temp_dir_at_start": false
}"""
with open('/content/stable-diffusion-webui/config.json','w') as f:
    f.write(config_content)
print("✅ 設定完了")

# ----------------------------
# 8) インストールサマリー
# ----------------------------
print("\n" + "="*60)
print("🎯 セットアップ完了サマリー:")
print("="*60)
model_dir = "/content/stable-diffusion-webui/models/Stable-diffusion"
if os.path.exists(model_dir):
    model_files = [f for f in os.listdir(model_dir) if f.endswith(('.safetensors','.ckpt'))]
    print("📁 インストールされたモデル:")
    if model_files:
        for mf in model_files:
            fp = os.path.join(model_dir, mf)
            try:
                sz = os.path.getsize(fp)/(1024*1024*1024)
                print(f"   ✅ {mf} ({sz:.2f} GB)")
            except Exception:
                print(f"   ✅ {mf}")
    else:
        print("   ❌ モデルが見つかりません")
else:
    print("   ❌ モデルディレクトリが見つかりません")

ext_dir = "/content/stable-diffusion-webui/extensions"
if os.path.exists(ext_dir):
    extensions = [d for d in os.listdir(ext_dir) if os.path.isdir(os.path.join(ext_dir, d))]
    print("\n🔌 インストールされた拡張機能:")
    for e in extensions: print(f"   ✅ {e}")
print("="*60 + "\n")

# ----------------------------
# 9) WebUI 起動（Python 3.10 で）+ URL検出 + 音声通知
# ----------------------------
print("\n🚀 Stable Diffusion WebUI を Python 3.10 で起動します")
%cd /content/stable-diffusion-webui

os.environ['TORCH_COMMAND'] = "pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121"

print("\n💡 メモ")
print("- 起動後、Share URLを検出したら音でお知らせします")
print("- 以降のインストール/実行は `micromamba run -n py310 ...` を使うと、Python 3.10 側で動きます")
print("- ノートブックの `!python -V` は 3.11/3.12 のままなので混同に注意")
print("\n起動コマンドを実行します...\n")

# ビープ音を生成して再生
def play_beep():
    """URL検出時に鳴らす通知音"""
    sample_rate = 44100
    duration = 0.5  # 0.5秒
    frequency = 880  # A5音（880Hz）
    t = np.linspace(0, duration, int(sample_rate * duration))
    audio_data = np.sin(2 * np.pi * frequency * t) * 0.3  # 音量を30%に
    display(Audio(audio_data, rate=sample_rate, autoplay=True))

# 起動コマンド
launch_cmd = [
    "micromamba","run","-n","py310","python","launch.py",
    "--share",
    "--listen",
    "--disable-console-progressbars",
    "--opt-sdp-attention",
    "--enable-insecure-extension-access",
    "--lora-dir","/content/stable-diffusion-webui/models/Lora",
    "--medvram-sdxl",
    "--no-half-vae",
    "--api",
    '--cors-allow-origins=*'
]

# プロセス起動
process = subprocess.Popen(
    launch_cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

# URL検出パターン
url_pattern = re.compile(r'Running on public URL: (https://[^\s]+)')
url_found = False

# 出力を監視
while True:
    line = process.stdout.readline()
    if not line:
        break
    line = line.rstrip()
    if line:
        print(line)
        
        # URLを検出
        if not url_found:
            match = url_pattern.search(line)
            if match:
                url = match.group(1)
                url_found = True
                
                # 通知音を鳴らす
                print("\n" + "="*70)
                print("🎉 WebUI が起動しました!")
                print("="*70)
                print(f"\n🌐 Share URL: {url}")
                display(HTML(f'<h2 style="color: #4CAF50;">🔗 <a href="{url}" target="_blank">{url}</a></h2>'))
                play_beep()
                print("\n" + "="*70 + "\n")